# Introduction

In this homework you will implement the transition based depensency parser for projective tree.

Take a look at the training file. It has the [CoNLL format](http://ufal.mff.cuni.cz/conll2009-st/task-description.html) format, including the following columns:

| ID | TOKEN | LEMMA | UNIV_POS | TREEBANK_POS | FEAT | HEAD | DEP_RELATION |
|----|-------|-------|----------|--------------|------|------|--------------|
| 1  | This | this | DET | DT |  _ | 0 | det|
| 2  | ... | ... | ... | ... |  ... | ... | ... |

**ID:**           (int) Index of a word in a sentence, starting from 1 <br>
**TOKEN**:        (str) The word itself <br>
**TREEBANK_POS**: (str) The POS tag of the word <br>
**HEAD**:         (int) Head of the dependent <br>
**DEP_RELATION**: (str) The label of the dependency relationship 

In this homework, we will not need the UNIV_POS and FEAT columns.

To help you with the homework, we provide the sanity_check.py script. However the script depends on the function you implement in this notebook, so it may not work by directly running in the console. You can copy the whole file content into the `Unit test` session, and run the check functions as you need.

# Packages

In [ ]:
import sys,re
import numpy as np
from sklearn import linear_model
from scipy import sparse
from collections import Counter

# Initialization

In [ ]:
MINIMUM_FEAT_COUNT=5

# dictionary of feature: index
# feature = a dictionary of feature name: value
feature_vocab={}

# reverse_features[index] = feature
reverse_features=[]

# dictionary of label: index, where
# label = SHIFT, LEFTARC_DEPENDENCY_LABEL, RIGHTARC_DEPENDENCY_LABEL
label_vocab={}

# reverse_labels[index] = label
reverse_labels=[]

# number/ID of features
fmax = 0

# Checking for Projectivity 

The transition based dependency parsing algorithm features linear time complexity, but it works only for projective tree. So the first step we need is to check if a tree is projective. If you need help, feel free to google for working solutions. Use the `check_projective` function from the sanity_check.py file to check your projective function.

In [ ]:
def is_projective(toks):
    """
    params: toks is a list of (idd, tok, pos, head, lab) for a sentence
    return True if and only if the sentence has a projective dependency tree
    """
    # Implement your code below
    raise NotImplementedError


# Creating an Oracle

The training data in the conll file contains the nodes and edges of the parsing tree: 
* The initial **wbuffer** is the nodes in the tree
* The deps is the edges in the tree wich structure like: <br>
    deps = {head1:{
                  (head1, child1):dependency_label1,
                  (head1, child2):dependency_label2
                 }
            head2:{
                  (head2, child3):dependency_label3,
                  (head2, child4):dependency_label4
                 }
            }
We need to translate each tree into a list of configurations and gold_transitions (label), so later on we can featurize configuration and apply machine learning models on it.

Rember to use the `sanity_check` to help you with the task.

In [ ]:
def perform_shift(wbuffer, stack, arcs,
                  configurations, gold_transitions):
    """
    perform the SHIFT operation
    """

    # Implement your code below
    # your code should:
    # 1. append the latest configuration to configurations
    # 2. append the latest action to gold_transitions
    # 3. update wbuffer, stack and arcs accordingly
    # hint: note that the order of operations matters
    # as we want to capture the configurations and transition rules
    # before making changes to the stack, wbuffer and arcs
    raise NotImplementedError


def perform_arc(direction, dep_label,
                wbuffer, stack, arcs,
                configurations, gold_transitions):
    """
    params:
        - direction: {"LEFT", "RIGHT"}
        - dep_label: label for the dependency relations
    Perform LEFTARC_ and RIGHTARC_ operations
    """

    # Implement your code below
    # your code should:
    # 1. append the latest configuration to configurations
    # 2. append the latest action to gold_transitions
    # 3. update wbuffer, stack and arcs accordingly
    # hint: note that the order of operations matters
    # as we want to capture the configurations and transition rules
    # before making changes to the stack, wbuffer and arcs
    raise NotImplementedError
    

def tree_to_actions(wbuffer, stack, arcs, deps):
    """
    params:
    wbuffer: a list of word indices, top of buffer is at the end of the list
    stack: a list of word indices, top of buffer is at the end of the list
    arcs: a list of (label, head, dependent) tuples

    Given wbuffer, stack, arcs and deps
    Return configurations and gold_transitions (actions)
    """

    # configurations:
    # A list of tuples of lists
    # [(wbuffer1, stack1, arcs1), (wbuffer2, stack2, arcs2), ...]
    # Keeps tracks of the states at each step
    configurations=[]

    # gold_transitions:
    # A list of action strings, e.g ["SHIFT", "LEFTARC_nsubj"]
    # Keeps tracks of the actions at each step
    gold_transitions=[]

    # Implement your code below
    # hint:
    # 1. configurations[i] and gold_transitions[i] should
    # correspond to the states of the wbuffer, stack, arcs
    # (before the action was take) and action to take at step i
    # 2. you should call perform_shift and perform_arc in your code
    
    raise NotImplementedError
            
    return configurations, gold_transitions

# Tree Parsing with Predictions

In [ ]:
def isvalid(stack, wbuffer, action):
    """
    Helper function that returns True only if an action is
    legal given the current states of the stack and wbuffer
    """
    if action == "SHIFT" and len(wbuffer) > 0:
        return True
    if action.startswith("RIGHTARC") and len(stack) > 1 and stack[-1] != 0:
        return True
    if action.startswith("LEFTARC") and len(stack) > 1 and stack[-2] != 0:
        return True

    return False


def action_to_tree(tree, predictions, wbuffer, stack, arcs):
    """
    params:
    tree:
    a dictionary of dependency relations (head, dep_label)
        {
            child1: (head1, dep_lebel1),
            child2: (head2, dep_label2), ...
        }

    predictions:
    a numpy column vector of probabilities for different dependency labels
    as ordered by the global variable reverse_labels
    predictions.shape = (1, total number of dependency labels)

    wbuffer: a list of word indices, top of buffer is at the end of the list
    stack: a list of word indices, top of buffer is at the end of the list
    arcs: a list of (label, head, dependent) tuples

    """
    global reverse_labels

    # Implement your code below
    # hint:
    # 1. the predictions contains the probability distribution for all
    # possible actions for a single step, and you should choose one
    # and update the tree only once
    # 2. some actions predicted are not going to be valid
    # (e.g., shifting if nothing is on the buffer)
    # so sort probs and keep going until we find one that is valid.
    raise NotImplementedError
  

# Get_oracle

In [ ]:
# ============================================================
# THE FOLLOWING CODE IS PROVIDED
# ============================================================
def get_oracle(toks):
    """
    Return pairs of configurations + gold transitions (actions)
    from training data
    configuration = a list of tuple of:
        - buffer (top of buffer is at the end of the list)
        - stack (top of buffer is at the end of the list)
        - arcs (a list of (label, head, dependent) tuples)
    gold transitions = a list of actions, e.g. SHIFT
    """

    stack = [] # stack
    arcs = [] # existing list of arcs
    wbuffer = [] # input buffer

    # deps is a dictionary of head: dependency relations, where
    # dependency relations is a dictionary of the (head, child): label
    # deps = {head1:{
    #               (head1, child1):dependency_label1,
    #               (head1, child2):dependency_label2
    #              }
    #         head2:{
    #               (head2, child3):dependency_label3,
    #               (head2, child4):dependency_label4
    #              }
    #         }
    deps = {}

    # ROOT
    stack.append(0)

    # initialize variables
    for position in reversed(toks):
        (idd, _, _, head, lab) = position

        dep = (head, idd)
        if head not in deps:
            deps[head] = {}
        deps[head][dep] = lab

        wbuffer.append(idd)

    # configurations:
    # A list of (wbuffer, stack, arcs)
    # Keeps tracks of the states at each step
    # gold_transitions:
    # A list of action strings ["SHIFT", "LEFTARC_nsubj"]
    # Keeps tracks of the actions at each step
    configurations, gold_transitions = tree_to_actions(wbuffer, stack, arcs, deps)
    return configurations, gold_transitions


# Bonus Question
def featurize_configuration(configuration, tokens, postags):
    """
    !EXTRA CREDIT!:
    Add new features here to improve the performance of the parser

    Given configurations of the stack, input buffer and arcs,
    words of the sentence and POS tags of the words,
    return some features
    """
    wbuffer, stack, arcs = configuration
    feats = {}

    feats["%s_%s" % ("len_buffer", len(wbuffer))] = 1
    feats["%s_%s" % ("len_stack", len(stack))] = 1

    # single-word features
    if len(stack) > 0:
        feats["%s_%s" % ("stack_word_1", tokens[stack[-1]])] = 1
        feats["%s_%s" % ("stack_tag_1", postags[stack[-1]])] = 1
        feats["%s_%s_%s" % ("stack_tag_word_1", tokens[stack[-1]], postags[stack[-1]])] = 1

    if len(stack) > 1:
        feats["%s_%s" % ("stack_word_2", tokens[stack[-2]])] = 1
        feats["%s_%s" % ("stack_tag_2", postags[stack[-2]])] = 1
        feats["%s_%s_%s" % ("stack_tag_word_2", tokens[stack[-2]], postags[stack[-2]])] = 1

    if len(wbuffer) > 0:
        feats["%s_%s" % ("buffer_word_1", tokens[wbuffer[-1]])] = 1
        feats["%s_%s" % ("buffer_tag_1", postags[wbuffer[-1]])] = 1
        feats["%s_%s_%s" % ("buffer_tag_word_1", tokens[wbuffer[-1]], postags[wbuffer[-1]])] = 1

    # word-pair features
    if len(stack) > 1:
        feats["%s_%s_%s_%s" % ("stack1_word_tag_stack2_tag", tokens[stack[-1]], postags[stack[-1]], postags[stack[-2]])] = 1

    return feats


def get_oracles(filename):
    """
    Get configurations, gold_transitions from all sentences
    """
    with open(filename) as f:
        toks, tokens, postags = [], {}, {}
        tokens[0] = "ROOT"
        postags[0] = "ROOT"

        # a list of all features for each transition step
        feats = []
        # a list of labels, e.g. SHIFT, LEFTARC_DEP_LABEL, RIGHTARC_DEP_LABEL
        labels = []

        for line in f:
            cols = line.rstrip().split("\t")

            if len(cols) < 2: # at the end of each sentence
                if len(toks) > 0:
                    if is_projective(toks): # only use projective trees
                        # get all configurations and gold standard transitions
                        configurations, gold_transitions = get_oracle(toks)

                        for i in range(len(configurations)):
                            feat = featurize_configuration(configurations[i], tokens, postags)
                            label = gold_transitions[i]
                            feats.append(feat)
                            labels.append(label)

                    # reset vars for the next sentence
                    toks, tokens, postags = [], {}, {}
                    tokens[0] = "ROOT"
                    postags[0] = "ROOT"
                continue

            if cols[0].startswith("#"):
                continue

            # construct the tuple for each word in the sentence
            # for each word in the sentence
            # idd: index of a word in a sentence, starting from 1
            # tok: the word itself
            # pos: pos tag for that word
            # head: parent of the dependency
            # lab: dependency relation label
            idd, tok, pos, head, lab = int(cols[0]), cols[1], cols[4], int(cols[6]), cols[7]
            toks.append((idd, tok, pos, head, lab))

            # feature for training to predict the gold transition
            tokens[idd], postags[idd] = tok, pos

        return feats, labels


def train(feats, labels):
    """
    Train transition-based parsed to predict next action (labels)
    given current configuration (featurized by feats)
    Return the classifier trained using the logistic regression model
    """
    global feature_vocab, label_vocab, fmax, reverse_labels, reverse_features

    lid = 0 # label ID
    D = len(feats) # each row of feats corresponds to a row in labels
    feature_counts = Counter()

    # build dictionary of labels
    for i in range(D):
        for f in feats[i]:
            feature_counts[f] += 1

        if labels[i] not in label_vocab:
            label_vocab[labels[i]] = lid
            lid += 1

    # build dictionary of features
    for f in feature_counts:
        if feature_counts[f] > MINIMUM_FEAT_COUNT and f not in feature_vocab:
            feature_vocab[f] = fmax
            fmax += 1

    # build reverse lookup for features and labels
    reverse_labels = [None]*lid
    for label in label_vocab:
        reverse_labels[label_vocab[label]] = label

    reverse_features = [None]*fmax
    for feature in feature_vocab:
        reverse_features[feature_vocab[feature]] = feature

    # X is a D-by-fmax matrix, where each row represents
    # features for a configuration / actions
    # Y is a list of labels for all configurations
    X = sparse.lil_matrix((D, fmax))
    Y = []
    for i in range(D):
        for f in feats[i]:
            if f in feature_vocab:
                fid = feature_vocab[f]
                X[i,fid] = 1
        Y.append(label_vocab[labels[i]])

    print ("Docs: ", D, "Features: ", fmax)
    log_reg = (linear_model.LogisticRegression(C=1.0, penalty='l2', multi_class="auto")
                           .fit(sparse.coo_matrix(X), Y))

    return log_reg


def parse(toks, logreg):
    """
    parse sentence with trained model and return correctness measure
    """
    tokens, postags = {}, {}
    tokens[0] = "ROOT"
    postags[0] = "ROOT"

    heads, labels = {}, {}

    wbuffer, stack, arcs = [], [], []
    stack.append(0)

    for position in reversed(toks):
        # featurization
        (idd, tok, pos, head, lab) = position
        tokens[idd] = tok
        postags[idd] = pos

        # keep track of gold standards for performance evaluation
        heads[idd], labels[idd] = head, lab

        # update buffer
        wbuffer.append(idd)

    tree = {}
    while len(wbuffer) >= 0:
        if len(wbuffer) == 0 and len(stack) == 0: break
        if len(wbuffer) == 0 and len(stack) == 1 and stack[0] == 0: break

        feats = (featurize_configuration((wbuffer, stack, arcs), tokens, postags))
        X = sparse.lil_matrix((1, fmax))
        for f in feats:
            if f in feature_vocab:
                X[0,feature_vocab[f]]=1

        predictions = logreg.predict_proba(X)
        # your function will be called here
        action_to_tree(tree, predictions, wbuffer, stack, arcs)

    # correct_unlabeled: total number of correct (head, child) dependencies
    # correct_labeled: total number of correctly *labeled* dependencies
    correct_unlabeled, correct_labeled, total = 0, 0, 0

    for child in tree:
        (head, label) = tree[child]
        if head == heads[child]:
            correct_unlabeled += 1
            if label == labels[child]: correct_labeled += 1
        total += 1

    return [correct_unlabeled, correct_labeled, total]


def evaluate(filename, logreg):
    """
    Evaluate the performance of a parser against gold standard
    """
    with open(filename) as f:
        toks=[]
        totals = np.zeros(3)
        for line in f:
            cols=line.rstrip().split("\t")

            if len(cols) < 2: # end of a sentence
                if len(toks) > 0:
                    if is_projective(toks):
                        tots = np.array(parse(toks, logreg))
                        totals += tots
                        print ("%.3f\t%.3f\t%s" % (totals[0]/totals[2], totals[1]/totals[2], totals))
                    toks = []
                continue

            if cols[0].startswith("#"):
                continue

            idd, tok, pos, head, lab = int(cols[0]), cols[1], cols[4], int(cols[6]), cols[7]
            toks.append((idd, tok, pos, head, lab))


# Training and testing

In [2]:
# ============================================================
# Run this cell to train the model
#
# NOTE: this takes a while to train, so you can change 
# the input to "train.projective.short.conll" to 
# train on only a subset of the data
#
# # ============================================================
feats, labels = get_oracles("train.projective.short.conll")
# feats, labels = get_oracles("train.projective.conll")
logreg = train(feats, labels)
evaluate("dev.projective.conll", logreg)

NameError: name 'get_oracles' is not defined

# Unit test